In [63]:
import numpy as np
from sklearn import datasets
import pandas as pd

In [64]:
iris = datasets.load_iris()

In [65]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [66]:
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [67]:
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')

In [68]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [69]:
df

,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a
5,b,d,a,a
6,a,c,a,a
7,a,c,a,a
8,a,b,a,a
9,a,c,a,a


In [70]:
y = pd.DataFrame(iris.target)
name = y.columns[-1]
target = y[name]

In [71]:
unused_features = set(df.columns)
build_tree(df, target, unused_features, 0)

Level  0
Count of  0  is 50
Count of  1  is 50
Count of  2  is 50
Current Entropy  is =  1.584962500721156
Splitting on feature  pw_labeled  with gain ratio  0.6996382036222091

Level  1
Count of  0  is 50
Current Entropy  is = 0.0
Reached leaf node

Level  1
Count of  1  is 10
Current Entropy  is = 0.0
Reached leaf node

Level  1
Count of  2  is 34
Current Entropy  is = 0.0
Reached leaf node

Level  1
Count of  1  is 40
Count of  2  is 16
Current Entropy  is =  0.863120568566631
Splitting on feature  pl_labeled  with gain ratio  0.4334099495621067

Level  2
Count of  1  is 1
Current Entropy  is = 0.0
Reached leaf node

Level  2
Count of  2  is 8
Current Entropy  is = 0.0
Reached leaf node

Level  2
Count of  1  is 39
Count of  2  is 8
Current Entropy  is =  0.6581912658132185
Splitting on feature  sl_labeled  with gain ratio  0.12674503775809332

Level  3
Count of  1  is 14
Current Entropy  is = 0.0
Reached leaf node

Level  3
Count of  1  is 2
Current Entropy  is = 0.0
Reached leaf n

Function "print_leaf" to print information when leaf node is reached

In [58]:
def print_leaf(y, level):
    print("Level ", level)
    values = set(y)
    for val in values:
        cnt = len(y[y == val])
        print("Count of ", val, " is", cnt)
    print("Current Entropy  is = 0.0")  # since entropy of pure node is 0
    print("Reached leaf node")
    print()

Function "print_info" to print information when splitting on best_feature is done

In [57]:
def print_info(y, best_feature, level, init_entropy, best_gain_ratio):
    print("Level ", level)
    values = set(y)
    for val in values:
        cnt = len(y[y == val])
        print("Count of ", val, " is", cnt)
    print("Current Entropy  is = ", init_entropy)
    print("Splitting on feature ", best_feature," with gain ratio ", best_gain_ratio)
    print()

Function "calc_entropy" to calculate entropy of feature

In [56]:
def calc_entropy(frequency):
    entropy = 0    # initialising entropy as zero
    size = sum(frequency)
    for fr in frequency:
        if (fr != 0):
            prob = float(fr / size)   # finding the probability of each value in feature
            entropy += (-prob * np.log2(prob))   # calculating the entropy
    return entropy

Function "initial_entropy" to calculate entropy before splitting

In [54]:
def initial_entropy(y):
    freq = []
    diff_op = set(y)
    # iterating to all the possible values in y
    # and finding the frequency of each possible output
    # and append it to freq[]
    for op in diff_op:
        cnt = len(y.loc[y == op])
        
        freq.append(cnt)
    entropy = calc_entropy(freq)  # calling calc_entropy to calculate entropy
    return entropy

Function "split_info" to calculate split info of a feature

In [52]:
def split_info(one_feature):
    total = len(one_feature)
    splitInfo = 0
    possible_values = set(one_feature)
    # iterating to all the possible values in one_feature
    # and finding the frequency of each possible value
    # and calculating the split info
    for value in possible_values:
        cnt = len(one_feature[one_feature == value])
        DjByD = cnt / total
        splitInfo += (-DjByD * np.log2(DjByD))
    return splitInfo

Function "calc_gain" to calculate gain of a feature

In [50]:
def calc_gain(y, size, diff_op):
    freq = []
    # iterating to all the possible values in y
    # and finding the frequency of each possible output
    # and append it to freq[]
    for op in diff_op:
        cnt = len(y.loc[y == op])
        freq.append(cnt)
    entropy = calc_entropy(freq)  # calculating the entropy of the feature
    wt_ent = float(sum(freq) / size) * entropy    # calculating the wighted info gain
    return wt_ent

Function "build_tree" is the main function which receives the data and output values and split on best feature on the basis of gain ratio and recursively call build_tree

In [46]:
def build_tree(df, y, unused_features, level):
    # base case:
    if(len(unused_features) == 0):  # if all the features are used then return
        return
    if(len(set(y)) == 1):           # if the subset has only one output value i.e., pure node then
        print_leaf(y, level)        # print the leaf node information
        return
    best_feature = ' '              # initialising best_feature as empty string
    best_gain_ratio = 0             
    feature_ent = 0
    splitInfo = 0
    init_entropy = initial_entropy(y)          # calculating the entropy before splitting 
    diff_op = set(y)                           # store the possible output values in diff_op
    # iterating to all the features 
    # to find the best feature 
    for feature in unused_features:     
        splitInfo = split_info(df[feature])    # calculating the split info of the feature
        possible_values = set(df[feature])     # store the possible feature values in possible_values
        feature_ent = 0
        # iterating to all the values in the feature
        for value in possible_values:
            feature_ent += calc_gain(y.loc[df[feature] == value], len(y), diff_op)   # calculating the gain of a feature
        gain_ratio = (init_entropy - feature_ent) / splitInfo    # calculating the gain ratio of the feature
        # if any feature has better gain ratio
        # then update the best_gain_ratio
        if(gain_ratio > best_gain_ratio):
            best_gain_ratio = gain_ratio
            best_feature = feature
    if(best_feature == ' '):
        best_feature = list(unused_features)[0]
    subset_df = []
    subset_y = []
    vals = set(df[best_feature])
    # iterating to all the features of best_feature
    # and storing the subset of df & y with f == val
    # in the list
    for val in vals:
        d_subset = df.loc[df[best_feature] == val]
        y_subset = y.loc[df[best_feature] == val]
        subset_df.append(d_subset)
        subset_y.append(y_subset)
    print_info(y, best_feature, level, init_entropy, best_gain_ratio)  # print the information of splitting
    unused_features.remove(best_feature)      # remove the best_feature from the unused_features
    # iterating to all the subsets of best_feature
    # and calling the build_tree recursively 
    # on each of the subset
    for i in range(len(subset_df)):
        build_tree(subset_df[i], subset_y[i], unused_features, level + 1)